In [ ]:
import lxml
import requests
import time
import sys
#import progressbar as PB

YOUTUBE_IN_LINK = 'https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&order=relevance&pageToken={pageToken}&videoId={videoId}&key={key}'
YOUTUBE_LINK = 'https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&order=relevance&videoId={videoId}&key={key}'
key = 'key'

def commentExtract(videoId):
    print ("Comments downloading")
    page_info = requests.get(YOUTUBE_LINK.format(videoId = videoId, key = key))
    while page_info.status_code != 200:
        if page_info.status_code != 429:
            print ("Comments disabled")
            sys.exit()

        time.sleep(20)
        page_info = requests.get(YOUTUBE_LINK.format(videoId = videoId, key = key))

    page_info = page_info.json()

    comments = []
    
    for i in range(len(page_info['items'])):
        comments.append(page_info['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal'])
        
    numberofpages=0   
   
    while numberofpages<25 and 'nextPageToken' in page_info:
        
        numberofpages=numberofpages+1
        temp = page_info
        page_info = requests.get(YOUTUBE_IN_LINK.format(videoId = videoId, key = key, pageToken = page_info['nextPageToken']))

        while page_info.status_code != 200:
            time.sleep(10)
            page_info = requests.get(YOUTUBE_IN_LINK.format(videoId = videoId, key = key, pageToken = temp['nextPageToken']))
        page_info = page_info.json()

        for i in range(len(page_info['items'])):
            comments.append(page_info['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal'])
           

    return comments

In [ ]:
comments=commentExtract("sHAkDTlv8fA")

In [ ]:
import pandas as pd
df = pd.DataFrame({'col':comments})
df.to_csv('YouTubeComments.csv')

In [ ]:
comments

### Cleaning the output

In [ ]:
comment = '\n'.join(comments)
import re
def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','USER', text)
    text = text.lower().replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()
data =preprocess_text(comment)

In [ ]:
data # after clean the output

In [ ]:
import pickle

with open('Comments_output', 'wb') as fp:
    pickle.dump(data, fp) # save to text file